In [1]:
from datasets import GTA5, CityScapes
from models.deeplabv2.deeplabv2 import get_deeplab_v2
import cv2
from PIL import Image
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from train import train
from utils import poly_lr_scheduler, fast_hist, per_class_iou

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ardaerendogru (aeddea). Use `wandb login --relogin` to force relogin


In [2]:
BATCH_SIZE = 6
NC=19
NUM_WORKERS = 8

def to_tensor_no_normalize(pil_image):
    # Apply the resize transformation to the image
    # Convert the PIL image to a tensor
    return torch.tensor(np.array(pil_image, dtype=np.uint8))

transform_cityscapes_image = transforms.Compose([
    transforms.Resize((256, 512)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_label = transforms.Compose([
    transforms.Resize((256, 512), interpolation=Image.NEAREST),
    transforms.Lambda(lambda img: to_tensor_no_normalize(img))
])
transform_cityscapes_label = transforms.Compose([
    transforms.Resize((256, 512), interpolation=Image.NEAREST),
    transforms.Lambda(lambda img: to_tensor_no_normalize(img))
])

cityscapes_train_dataset = CityScapes('./Cityscapes', 'train', transform_image=transform_cityscapes_image , transform_label=transform_cityscapes_label)
cityscapes_test_dataset = CityScapes('./Cityscapes', 'val', transform_image=transform_cityscapes_image , transform_label=transform_cityscapes_label)
GTA5_dataset = GTA5('./GTA5', transform_image=transform_cityscapes_image , transform_label=transform_cityscapes_label)


cityscapes_train_dataloader = DataLoader(cityscapes_train_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)
cityscapes_test_dataloader = DataLoader(cityscapes_test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)
GTA5_dataloader = DataLoader(GTA5_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)



## 2.1 DeepLabV2

In [ ]:

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=255)
init_lr = 0.001 #0.0001
model = get_deeplab_v2(num_classes=19, pretrain=True, pretrain_model_path='./models/deeplab_resnet_pretrained_imagenet.pth').cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=init_lr)

deeplab_result = train( model,
                        loss_fn, optimizer,
                        cityscapes_train_dataloader,
                        cityscapes_test_dataloader, 
                        50,
                        'step2_DeepLabV2')


## 2.2 BiseNet


In [ ]:
from tqdm.notebook import tqdm
from models.bisenet.build_bisenet import BiSeNet
bisenet_model = BiSeNet(20, 'resnet101').cuda()
from utils import poly_lr_scheduler, fast_hist, per_class_iou
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=255)
init_lr = 0.001 #0.0001
optimizer = torch.optim.Adam(bisenet_model.parameters(), lr=init_lr)
result2_2 = train(bisenet_model, loss_fn, optimizer,cityscapes_train_dataloader,cityscapes_test_dataloader, 50,'step2_BiseNet')


## 3.1 Bisenet

In [ ]:
from tqdm.notebook import tqdm
from models.bisenet.build_bisenet import BiSeNet
bisenet_model = BiSeNet(20, 'resnet101').cuda()
from utils import poly_lr_scheduler, fast_hist, per_class_iou
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=255)
init_lr = 0.001 #0.0001
optimizer = torch.optim.Adam(bisenet_model.parameters(), lr=init_lr)
result3_1 = train(bisenet_model, loss_fn, optimizer,GTA5_dataloader,cityscapes_test_dataloader, 50,'step3_BiseNet')


## 3.2 Bisenet with augmentation



In [3]:
import albumentations as A
from tqdm.notebook import tqdm
from models.bisenet.build_bisenet import BiSeNet
bisenet_model = BiSeNet(20, 'resnet101').cuda()
from utils import poly_lr_scheduler, fast_hist, per_class_iou
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=255)
init_lr = 0.025 #0.0001
optimizer = torch.optim.Adam(bisenet_model.parameters(), lr=init_lr)

transform = A.Compose([
    A.Resize(256, 512),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    A.HueSaturationValue(p=0.5),
    A.GaussianBlur(kernel_size=(5, 5), sigma=(0.1, 1), p=0.5),
    A.RandomFog(p=0.5),
    A.GaussNoise(p=0.5),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),

])
GTA5_dataset = GTA5('./GTA5',augmentations=transform)
GTA5_dataloader = DataLoader(GTA5_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

result3_1 = train(bisenet_model, loss_fn, optimizer,GTA5_dataloader,cityscapes_test_dataloader, 50,'step3_BiseNet')

  0%|          | 0/50 [00:00<?, ?it/s]

Epoch: 0
Train Loss: 0.9527229992605799, Validation Loss: 1.3393097122510274
Train mIoU: 0.11397026098410527, Validation mIoU: 0.1179773625920738
Epoch: 1
Train Loss: 0.8151436878336991, Validation Loss: 1.345504036971501
Train mIoU: 0.1493951902184583, Validation mIoU: 0.11246220916350894
Epoch: 2
Train Loss: 0.7690537557613364, Validation Loss: 1.3212483071145558
Train mIoU: 0.1633676347937043, Validation mIoU: 0.10447827280133798
Epoch: 3
Train Loss: 0.7305930418362149, Validation Loss: 1.1998044713622047
Train mIoU: 0.17433308347725895, Validation mIoU: 0.11885606892088423
Epoch: 4
Train Loss: 0.6981318351462019, Validation Loss: 1.144949456055959
Train mIoU: 0.18468218746683168, Validation mIoU: 0.13604399105600132
Epoch: 5
Train Loss: 0.6669123577271148, Validation Loss: 1.166615374031521
Train mIoU: 0.1934911631200699, Validation mIoU: 0.13466933298964942
Epoch: 6
Train Loss: 0.6447407729048237, Validation Loss: 1.1015621225039165
Train mIoU: 0.20150999430993394, Validation mIoU

KeyboardInterrupt: 

In [ ]:
# transform = A.Compose([
#     A.Resize(256, 512),
#     A.HorizontalFlip(p=0.5),
#     A.Rotate(limit=5, p=0.5,border_mode=1),
#     A.GaussianBlur(kernel_size=(5, 5), sigma=(0.1, 1), p=0.5),
#     A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
# ])

# img_path = './Cityscapes/Cityspaces/images/train/hanover/hanover_000000_000164_leftImg8bit.png'
# label_path = './Cityscapes/Cityspaces/gtFine/train/hanover/hanover_000000_000164_gtFine_labelTrainIds.png'
# img = np.array(Image.open(img_path))
# label = np.array(Image.open(label_path))
# transformed = transform(image=img, mask=label)
# img = transformed['image']
# label = transformed['mask']
# Image.fromarray(img)
